In [13]:
import ngsolve as ngs
from ngsolve.webgui import Draw

from createGeom import makeGeo
geoOCC = makeGeo()

ngmesh = geoOCC.GenerateMesh()
mesh = ngs.Mesh(ngmesh)

mesh.Refine()
# mesh.Refine()
mesh.Curve(7)

print(mesh.ne)

dsadas313
4944


In [12]:
39552/8

4944.0

In [3]:
# from bhdata import BHCurves
# fun_dw  = BHCurves(4)
# fun_w   = fun_dw.Integrate()
# fun_ddw = fun_dw.Differentiate()

from bhdata import Brauer, BrauerCut
fun_w, fun_dw, fun_ddw  = BrauerCut()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus"
nonlinear = "stator"

strom = 1e5
amps = 0.025**2*3.15*strom
print(f'Applying a current of {amps:.2f} Amps')

J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))
# Draw(J, mesh)

Applying a current of 196.88 Amps


In [4]:
# print(HCurl.ndof)
# print(mesh.GetMaterials())

deg = 3

HCurl = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face')
# HCurl = ngs.HCurl(mesh, order = deg, nograds = True)
u,v = HCurl.TnT()


# Nonlinear:
maxit = 2000
tol2 = 1e-8
regb = 1e-8

A = ngs.GridFunction(HCurl)
B = ngs.curl(A)
normB = ngs.sqrt(B*B + regb)


cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B).Compile()

def fun_W():
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - J*A, mesh)
    # with ngs.TaskManager(): res = ngs.Integrate(cf_energy - J*A, mesh, order = 2*deg)
    # with ngs.TaskManager(): res = ngs.Integrate(cf_energy - ngs.curl(Hs)*A, mesh)
    # print("res:" + str(ngs.Integrate(cf_energy, mesh)))
    return res

ir = ngs.IntegrationRule(ngs.fem.ET.TET, order = 2*deg)


cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()

rhs = ngs.LinearForm(HCurl)
rhs += ngs.SymbolicLFI(cf_rhs*B*ngs.curl(v) - J*v, intrule = ir)

# rhs = ngs.LinearForm((cf_rhs*B*ngs.curl(v) - ngs.curl(v)*Hs)*ngs.dx)
# rhs = ngs.LinearForm((cf_rhs*B*ngs.curl(v) - ngs.curl(Hs)*v)*ngs.dx)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs


Id = ngs.CF((1,0,0,
             0,1,0,
             0,0,1), dims=(3,3))

BBt = ngs.CF((B[0]*B[0], B[0]*B[1], B[0]*B[2],
              B[1]*B[0], B[1]*B[1], B[1]*B[2],
              B[2]*B[0], B[2]*B[1], B[2]*B[2]), dims = (3,3))


fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id).Compile()

K_iter = ngs.BilinearForm(HCurl)
K_iter += ngs.SymbolicBFI(cf_iter*ngs.curl(u)*ngs.curl(v), intrule = ir)
# K_iter += (cf_iter*ngs.curl(u)*ngs.curl(v))*ngs.dx
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter


In [5]:
from ttictoc import tic, toc
from ngsolve.krylovspace import CGSolver

from ngcotree import *
newFreeDofs = CoTreeBitArray(mesh, HCurl, plot = False)

print("Using 3D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,HCurl.ndof, "DOFs.\n ")

with ngs.TaskManager(): A.Set(ngs.CF((0,0,0)))

du = ngs.GridFunction(HCurl)
uo = ngs.GridFunction(HCurl)
wo = 1e12

for it in range(1,maxit+1):
    tic()
    # with ngs.TaskManager():
    #     K_iter.Assemble()
    #     rhs.Assemble()
    #     res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()

    
    # iterativeSolver = CGSolver(K_iter.mat, freedofs = HCurl.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
    # iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-8,  maxiter = maxit*10)
    with ngs.TaskManager():
        iterativeSolver = CGSolver(K_iter.mat, freedofs = newFreeDofs, tol  = 1e-8,  maxiter = maxit, printrates = True)

        # du.vec.data = iterativeSolver * dw.vec
        du.vec.data = da.mat.Inverse(newFreeDofs, inverse="sparsecholesky") * dw.vec 
    
    # print('MAXdu: ' + str(du.vec.FV().NumPy().max()))
    # print('MAXdw: ' + str(dw.vec.FV().NumPy().max()))

    if len(iterativeSolver.residuals) == maxit: print("... Failure!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()
    
    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm
    
    # wn = 1e12
    if abs(wo-w) < tol2:
    # if abs(wn-w) < tol2:
    # if nrm/nrm0 < tol2:
        # print(wo,w)
        print("converged to desired tolerance")
        break
    # elif abs(wo-w) < tol2*1e-2:
    #     print("stopped early due to stagnation")
    #     break
    else:
        # linesearch
        # print("Doing LS:")
        uo.vec.data = A.vec.data
        wo = w
        alpha = 1
        for init in range(1,2100):
            A.vec.data -= alpha*du.vec.data
            wn = fun_W()
            # print(w,wn)
            if wn < w - alpha*0.1*nrm:
                # print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,len(iterativeSolver.residuals),alpha,w,nrm/nrm0))
                break
            else:
                # print(alpha)
                alpha = alpha/2
                A.vec.data = uo.vec.data

Using 3D mesh with ne= 39552 elements and nv= 8459 points and  592207 DOFs.
 
Iter:  1 | assem : 3.40s | CG took 6.87s with    0 iterations | alpha : 0.50 | energy = 0.0000000007 | relres = 1.00e+00 |
Iter:  2 | assem : 3.59s | CG took 6.63s with    0 iterations | alpha : 0.50 | energy = -0.1157972168 | relres = 8.21e-02 |
Iter:  3 | assem : 3.68s | CG took 7.28s with    0 iterations | alpha : 1.00 | energy = -0.1235285987 | relres = 3.27e-03 |
Iter:  4 | assem : 3.47s | CG took 6.85s with    0 iterations | alpha : 1.00 | energy = -0.1240942101 | relres = 2.33e-04 |
Iter:  5 | assem : 3.51s | CG took 6.75s with    0 iterations | alpha : 1.00 | energy = -0.1241344667 | relres = 5.89e-06 |
Iter:  6 | assem : 3.59s | CG took 7.33s with    0 iterations | alpha : 1.00 | energy = -0.1241354410 | relres = 1.66e-08 |
converged to desired tolerance


In [6]:
# !pip install webgui_jupyter_widgets
# clipping = { "function" : True,  "pnt" : (2.5,1.5,1), "vec" : (0,1,-1) }
# Draw (func, mesh, );
# Draw(A, mesh)

# clipping = { "function" : True}
# Draw (ngs.curl(A), mesh, clipping = True, objects = {"Clipping Plane": True});

B = ngs.curl(A)

clipping = { "function" : True, "z" : -0.4}
# clipping = { "function" : True, "pnt" : (0,0,0.1), "vec" : (0,0,-1)}
Draw (B, mesh, clipping = clipping);

normB = ngs.sqrt(B*B)
H = B*mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()

Draw (H, mesh, clipping = clipping);
# Draw(ngs.curl(A), mesh)
# Draw(ngs.curl(A), mesh, settings = {"Colormap" : { "ncolors" : 32 }})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [9]:
scalars = mesh.MaterialCF({"coil_plus|coil_minus": 0, "stator": 0.1}, default = 0)
L2 = ngs.L2(mesh, order = 0)
scal = ngs.GridFunction(L2)
scal.Set(scalars)

vtk = ngs.VTKOutput(mesh, coefs = [B,H,scal],names = ["B","H","scalars"], filename = "B2", order = 2)
vtk.Do()

'B2'

In [8]:
import pyvista as pv
pvmesh = pv.read("B2.vtu")
pvmesh.set_active_scalars("scalars")
pvmesh.set_active_vectors("B")
threshed = pvmesh.threshold([0.8,2.1])
# threshed.set_active_vectors("B")

p = pv.Plotter()
p.add_mesh(threshed, style = 'surface', opacity = 0.1, label = None)

arrows = pvmesh.glyph(scale = "B", orient = True, tolerance = 0.01, factor = 0.01)
# arrows = pvmesh.glyph(scale = "B", orient = True, tolerance = 0.01, factor = 0.01)
p.add_mesh(arrows)

# pvmesh.set_active_vectors("J")
# arrows2 = pvmesh.glyph(scale = "J", orient = True, tolerance = 0.001, factor = 1*1e-8)
# p.add_mesh(arrows2, color = "black")

p.show(jupyter_backend = "html")
# p.show()

ValueError: Empty meshes cannot be plotted. Input mesh has zero points. To allow plotting empty meshes, set `pv.global_theme.allow_empty_mesh = True`